# Adult UCI dataset

Classification with preprocessing and pipeline

In this example we will use the [Adult UCI dataset](https://archive.ics.uci.edu/dataset/2/adult)

The target is the `income` boolean column, specifying if the individual income si either *low* or *high*.

In this example we will do simply the classification with hold-out test and no optimisation. The students are invited to expand the exercise with a GridSearchCV optimisation and several classifiers.

## Technical aspects
In `sklearn` a `Pipeline` is a sequence of operation such that the output of one operation is the input to the following. Each operation is a transformation of data.

Examples of those operations are:
- column transformations:
  - encodings:
    - `OneHotEncoding` to transform a categorical column into a set of `0-1` columns
    - `OrdinalEncoding` to transform an ordinal column into a numeric one
  - imputation of null values, such as `SimpleImputer`, this one requires a **strategy**, for example, in numbers the nulls can be filled with the `median`, in categoricals the nulls can be filled with a constant value, such as `unknown`
  - numeric transformations, such as:
    - `MinMaxScaler`
    - `StandardScaler`
- estimators

In `sklearn` we have available the `ColumnTransformer` to transform groups of columns in a uniform way. It requires, for each group, a transformation pipeline and a list of attributes to be transformed with that pipeline

We will then build a final pipeline composed by the preprocessing with the `ColumnTransformer` and the classifier

## Workflow
1. load the file `adults.csv` into a dataframe and explore it
1. use as target the column `income`, separate the predicting columns and the target into `X` and `y`
1. show the percentage of null values in each column
1. prepare the variable `categorical_features` containing all the names of the categorical features of `X` excluding `education` (it will be ignored) and `sex` (it will be transformed separately);
1. prepare the variable `numeric_features` containing all the names of the numeric features excluding `fnlwgt` (this is the *importance* of each observation, it can be used in the final classification report as `sample_weight`)
1. the target should be binary, inspecting the unique values you will see that there are four distinct values due to typing errors, reduce the target to binary with an appropriate mapping (hint: you can use the `.map()` function of Pandas
1. split into train and test
1. prepare the column transformers for:
  1. numeric: simple imputation with the median and standard scaling
  1. categorical: simple imputation with nulls substituted by `unknown`
  1. boolean: ordinal encoding
1. prepare a pipeline with:
  1. the column transformer as preprocessor
  1. the `DecisionTreeClassifier`
1. fit the pipeline to the train part
1. predict the test and produce a classification report

In [1]:
import pandas as pd

df = pd.read_csv('../data/adult/adults.csv', sep=',')
df.describe(include='all')

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
count,48842.000000,47879,4.884200e+04,48842,48842.000000,48842,47876,48842,48842,48842,48842.000000,48842.000000,48842.000000,48568,48842
unique,NaN,9,NaN,16,NaN,7,15,6,5,2,NaN,NaN,NaN,42,4
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,33906,NaN,15784,NaN,22379,6172,19716,41762,32650,NaN,NaN,NaN,43832,24720
mean,38.643585,NaN,1.896641e+05,NaN,10.078089,NaN,NaN,NaN,NaN,NaN,1079.067626,87.502314,40.422382,NaN,NaN
std,13.710510,NaN,1.056040e+05,NaN,2.570973,NaN,NaN,NaN,NaN,NaN,7452.019058,403.004552,12.391444,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.175505e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.781445e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.376420e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


In [2]:
df.isna().sum() / df.shape[0] * 100 # percentage of missing values

age               0.000000
workclass         1.971664
fnlwgt            0.000000
education         0.000000
education-num     0.000000
marital-status    0.000000
occupation        1.977806
relationship      0.000000
race              0.000000
sex               0.000000
capital-gain      0.000000
capital-loss      0.000000
hours-per-week    0.000000
native-country    0.560993
income            0.000000
dtype: float64

In [3]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

In [4]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype("category")

df.dtypes

age                  int64
workclass         category
fnlwgt               int64
education         category
education-num        int64
marital-status    category
occupation        category
relationship      category
race              category
sex               category
capital-gain         int64
capital-loss         int64
hours-per-week       int64
native-country    category
income            category
dtype: object

In [20]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

random_state = 346
np.random.seed(random_state)

In [6]:
target = 'income'

X = pd.DataFrame(df.drop(columns=[target]))
y = pd.Series(df[target])

X.shape, y.shape

((48842, 14), (48842,))

In [7]:
categorical_features = X.select_dtypes(exclude=["number","bool_","object_"]).columns.drop(['education', 'sex'])
categorical_features.to_list()

['workclass',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'native-country']

In [8]:
numeric_features = X.select_dtypes(include=np.number).columns.drop(['fnlwgt'])
numeric_features.to_list()

['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [9]:
y = y.map(lambda x: str(x).replace('.', ''))

In [10]:
y.value_counts()

income
<=50K    37155
>50K     11687
Name: count, dtype: int64

### Here you should prepare the numeric, boolean and categorical transformers

`your_transformer = Pipeline(
  steps = [
              ("step name", transformer )
            , ("step name", transformer)
          ]
)`

In [11]:
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("minmax", MinMaxScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)


boolean_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("ordinal", OrdinalEncoder()),
    ]
)

### Here you collect the transformers in the `ColumnTransformer`

In [12]:
# preprocessor = ColumnTransformer(
#     transformers=[
#           ("transformer name", transformer, list_of_features)
#         , ("transformer name", transformer, list_of_features)
#         , ("transformer name", transformer, list_of_features)
#     ]
#     , sparse_threshold=0 # this prevents internal representation in sparse matrices
#                          # it is useful to speed operations
# )

In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("bool", boolean_transformer, ['sex'])
    ],
    sparse_threshold=0  # this prevents internal representation in sparse matrices
    # it is useful to speed operations
)

### Final operations

- prepare the final pipeline composed by the preprocessor and the classifier
- do the train/test split
- fit the pipeline to the train part
- predict the test part
- produce the classification report

In [14]:
# clf = Pipeline(
#     steps=[
#           ("step name", component)
#         , ("step name", component)
#         ]
# )

In [15]:
clf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("feature_selection", SelectPercentile(chi2)),
        ("classifier", DecisionTreeClassifier())
    ])

In the classification report use the parameter `sample_weight` passing the column `fnlwgt`

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_state, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((32724, 14), (16118, 14), (32724,), (16118,))

In [18]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('minmax',
                                                                   MinMaxScaler())]),
                                                  Index(['age', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(f...
                                                  Index(['workclass', 'marital-status', 'occupation', 'relationship', 'race',
       'native-country'],
      dtype='object')),
                                                 ('bool',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['sex'])])),
                ('feature_selection',
                 SelectPercentile(score_func=<function chi2 at 0x171c25940>)),
                ('classifier', DecisionTreeClassifier())])

In [19]:
y_test_pred = clf.predict(X_test)

In [21]:
print(classification_report(y_test, y_test_pred, sample_weight=X_test['fnlwgt']))

              precision    recall  f1-score   support

       <=50K       0.87      0.94      0.90 2340621490.0
        >50K       0.75      0.54      0.63 730444685.0

    accuracy                           0.85 3071066175.0
   macro avg       0.81      0.74      0.77 3071066175.0
weighted avg       0.84      0.85      0.84 3071066175.0

